In [1]:
import os
import pandas as pd
import pandasql as ps
import numpy as np
from datetime import datetime

#-- Data names
# directory
dir="/mnt/f/Tirocinio_HDD/Data/ADNI/"
#files
filename=dir+'FilesMerged.xlsx'


adni_data_summary=dir+"ADNI_data_summary_02-05-2022.xlsx"
UPENNBIOMK1=dir+"UPENNBIOMK9_04_19_17.csv"
UPENNBIOMK2=dir+"UPENNBIOMK10_07_29_19.csv"
UPENNBIOMK3=dir+"UPENNBIOMK12_01_04_21.csv"
fMRI=dir+"fMRI_SubjID_info_V2.xlsx"
dfMRI=pd.read_excel(fMRI, sheet_name='Main')

adni=pd.read_excel(adni_data_summary)
upenn1=pd.read_csv(UPENNBIOMK1)
upenn2=pd.read_csv(UPENNBIOMK2)
upenn3=pd.read_csv(UPENNBIOMK3)
adni=adni[adni['FMRI PREPROCESSING']==1]
upenn1=upenn1[upenn1.columns[upenn1.columns.isin(['RID','EXAMDATE','ABETA','PTAU','TAU'])]]
upenn2=upenn2[upenn2.columns[upenn2.columns.isin(['RID','DRAWDATE','ABETA42','PTAU','TAU'])]]
upenn3=upenn3[upenn3.columns[upenn3.columns.isin(['RID','EXAMDATE','PTAU','TAU','ABETA'])]]
adni=adni.dropna(subset=['RID'])
adni=adni.astype({'RID':'int'})
adni.reset_index(inplace=True)
adni=adni[adni.columns[adni.columns.isin(['ID','RID','ADNIMERGE','ACQUISITION DATE','AGE','SEX'])]]
upenn2=upenn2.rename(columns={"DRAWDATE":"EXAMDATE","ABETA42":"ABETA"})

In [2]:
adni


,ID,RID,ADNIMERGE,ACQUISITION DATE,AGE,SEX
0,002_S_1155,1155,LMCI,2017-04-24,68.2,M
1,002_S_1280,1280,CN,2017-03-13,80.8,F
2,002_S_4213,4213,CN,2017-08-14,84.0,F
3,002_S_4225,4225,CN,2018-11-12,77.1,M
4,002_S_4229,4229,LMCI,2017-09-20,72.4,M
...,...,...,...,...,...,...
783,114_S_6595,6595,AD,2018-11-16,79.8,M
784,123_S_6891,6891,AD,2021-11-01,79.4,M
785,135_S_6840,6840,AD,2019-12-05,68.6,F
786,301_S_6592,6592,AD,2018-10-23,71.9,M


In [3]:
upenns=pd.concat([upenn1,upenn2,upenn3])
upenns=upenns.reset_index(drop=True)   

In [4]:
len(upenn1)+len(upenn2)+len(upenn3) #3115
len(upenns) #ok se 3115, OK

3115

In [5]:
date_adni=pd.to_datetime(adni.loc[0]['ACQUISITION DATE'],format='%Y-%m-%d') 
print(date_adni)
print(adni.loc[0])

2017-04-24 00:00:00
ID                           002_S_1155
RID                                1155
ADNIMERGE                          LMCI
ACQUISITION DATE    2017-04-24 00:00:00
AGE                                68.2
SEX                                   M
Name: 0, dtype: object


In [6]:
adniout=adni.copy()

adniout['ABETA']="-99"
adniout['TAU' ]="-99"
adniout['PTAU']="-99"
adniout['EXAMDATE']="-99"
adniout['DIFF DATE']=-99
for index in range (0,len(adni)):
    if adni.loc[index]['RID'] in upenns['RID'].values:
        #print(adni.loc[index])
        adni_index=adni.loc[index]['RID']
        tmp=upenns[upenns['RID'].values == adni.loc[index]['RID']] #tutti i record con lo stesso RID, ora bisogna filtrare per data
        tmp=tmp.reset_index(drop=True) 
        #print(tmp)
        #filtra per data
        #ciclo su tutti gli elementi del sottoinsime RID del momento e scelgo quello con la data più vicina al record presente in adni
        date_adni=pd.to_datetime(adni.loc[index]['ACQUISITION DATE'],format='%Y-%m-%d') 
        #print("date adni:"+str(date_adni))
        idx_max_date = 0 #all'inizio assumo che sia la prima per semplicità
        diff=999999 #inizializzo la minima differenza
        for e in range(0,len(tmp)):
            line_e_date_tmp=datetime.strptime(tmp.loc[e]['EXAMDATE'],'%Y-%m-%d')
            #print(line_e_date_tmp)
            tdiff=abs(date_adni-line_e_date_tmp).days
            #print("DIFFERENZA: "+str(tdiff))
            if diff < tdiff:
                #do nothing
                #print("La minore rimane :" + str(diff))
                diff=diff
            else:
                #diff vecchia > diff del momento
                diff=tdiff
                idx_max_date=e #indice della riga con la minore differenza di giorni
                #print(tmp.loc[e]['EXAMDATE'])
                #print(diff)
        #print("#####")
        #print("Migliore differenza: "+str(diff))
       # print(tmp.loc[idx_max_date])
        adniout.at[index,'ABETA']=tmp.loc[idx_max_date]['ABETA']
        adniout.at[index,'TAU' ]=tmp.loc[idx_max_date]['TAU']
        adniout.at[index,'PTAU']=tmp.loc[idx_max_date]['PTAU']
        adniout.at[index,'EXAMDATE']=tmp.loc[idx_max_date]['EXAMDATE']
        adniout.at[index,'DIFF DATE']=diff
        #print(str(tmp.loc[idx_max_date])+" ----> "+str(diff))
        #print("---- VS ----")
        #print(adni.loc[index])
        #print("################################")
        
subjs_skipped=adniout[adniout.EXAMDATE == "-99"]        
adniout=adniout[adniout.EXAMDATE != "-99"]  
adniout.sort_values('ADNIMERGE',inplace=True)
adniout=adniout.reset_index()
subjs_skipped=subjs_skipped.reset_index()   
      

In [7]:
# >1700 a int

In [8]:
#### II PART ####
#quanti sono i volumi fMRI per ogni soggetto
adniout['fMRI Volumes']=0
dfMRI=dfMRI[dfMRI.columns[dfMRI.columns.isin(['Subj ID','Volumes'])]]
dfMRI['RID']=0
dfMRI=dfMRI.fillna(0) #ci sono svariati Nan, li rimpiazzo con 0 per semplicità 

In [9]:
#extract RIDs from new file
for i in range(0,len(dfMRI)):
    ridTMP=dfMRI.loc[i]['Subj ID']
    ridTMP=ridTMP[6:]
    dfMRI.at[i,'RID']=ridTMP


In [10]:
#assegna i volumi presenti ai soggetti che combaciano
for line in range(0,len(adniout)):
    rid=adniout.loc[line]['RID'] #rid del soggetto alla riga n
    rid_new_file=dfMRI[dfMRI['RID']==rid] #estrai soggetto con rid corrispondente
    if not rid_new_file.empty: #se l'intersezione dei due non è vuota 

        try: #devo usare questo perchè ci sono anche qui valori doppi tipo il 6294
            adniout.at[line,'fMRI Volumes']=rid_new_file['Volumes'] #assegno valore volumi 
        except:
            rid_new_file=rid_new_file.sort_values('Volumes',ascending=False).iloc[0]
            adniout.at[line,'fMRI Volumes']=rid_new_file['Volumes'] #assegno valore volumi 
            print(rid_new_file)
            #print("-------------------")
            #print(adniout.loc[line])
            print("###################")

Subj ID    032_S_6294
Volumes         197.0
RID              6294
Name: 208, dtype: object
###################


In [11]:
# Create a Pandas Excel writer using XlsxWriter as the engine.

writer = pd.ExcelWriter(filename, engine='xlsxwriter')
# Convert the dataframe to an XlsxWriter Excel object.
adniout.to_excel(writer, sheet_name='MAIN')
subjs_skipped.to_excel(writer, sheet_name='SUBJECTS_SKIPPED')


counts_skipped=subjs_skipped['ADNIMERGE'].value_counts()
counts_adni=adniout['ADNIMERGE'].value_counts()


#print(counts_adni)
#print(counts_skipped)

names=adniout['ADNIMERGE'].unique()
classes =['CLASSES ADNIOUT','CLASSES SKIPPED SUBJECTS']
#countsTOTAL=pd.DataFrame([np.repeat(0, [len(names)], axis=0).tolist(),np.repeat(0, [len(names)], axis=0).tolist()],classes,names)
dat=np.repeat(0, [len(names)], axis=0).tolist()
countsTOTAL=pd.DataFrame([dat,dat],classes,names)
for name in names:
    countsTOTAL.at['CLASSES ADNIOUT',name]=int(counts_adni[name])
    countsTOTAL.at['CLASSES SKIPPED SUBJECTS',name]=int(counts_skipped[name])
    
print(countsTOTAL)
countsTOTAL.to_excel(writer,sheet_name='CLASSES')

#complete=adnibackup[adniout['fMRI Volumes'] > 0] #aggiungo un foglio con i soggetti che hanno effettivamente tutto
#complete=adnibackup[adniout['fMRI Volumes'] > 0] #aggiungo un foglio con i soggetti che hanno effettivamente tutto
#complete.to_excel(writer,sheet_name='BACKUP')
#adnibackup.to_excel(writer,sheet_name='BACKUP')
# Close the Pandas Excel writer and output the Excel file.
writer.save()

                          AD   CN  EMCI  LMCI  SMC
CLASSES ADNIOUT           29  159   112    86  129
CLASSES SKIPPED SUBJECTS  39   94    32    40   30


In [12]:
# AMY-PET (AV45/FBB), Tau-PET (AV1451) 

In [13]:

print(adniout['fMRI Volumes'].value_counts().sort_values(ascending=False))

197    358
200    148
194      2
215      1
0        1
150      1
193      1
195      1
160      1
185      1
Name: fMRI Volumes, dtype: int64


In [14]:
countsTOTAL

,AD,CN,EMCI,LMCI,SMC
CLASSES ADNIOUT,29,159,112,86,129
CLASSES SKIPPED SUBJECTS,39,94,32,40,30
